# MIR

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import MIR
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    confusion_matrix_metrics,
    forgetting_metrics,
    timing_metrics,
    cpu_usage_metrics,
    gpu_usage_metrics,
    disk_usage_metrics,
    ram_usage_metrics,
    loss_metrics,
    class_accuracy_metrics,
    accuracy_matrix_metrics
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.plugins import WeightStoragePlugin

# Constants

In [5]:
TORCH_DEVICE = "cuda" if cuda.is_available() else "cpu"
N_EXPERIENCES = 20
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"class_saturation/{N_EXPERIENCES}"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/mir.pkl"

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

In [8]:
best_hyperparameters = {
    'mem_size': 50,
    'subsample': 5
}

## Evaluation metrics definition

In [9]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    loss_metrics(minibatch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    cpu_usage_metrics(minibatch=True, experience=True, stream=True),
    gpu_usage_metrics(experience=True, stream=True, gpu_id=TORCH_DEVICE),
    disk_usage_metrics(experience=True, stream=True),
    ram_usage_metrics(experience=True, stream=True),
    class_accuracy_metrics(experience=True, stream=True),
    accuracy_matrix_metrics(),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(stream=True, num_classes=scenario.n_classes),
    
    loggers=[InteractiveLogger()]
)

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")
/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/avalanche/evaluation/metrics/gpu_usage.py:56: UserWarning: Your system has no GPU!
  warnings.warn("Your system has no GPU!")


## Plugin defitinitions

In [10]:
model_plugins = [WeightStoragePlugin()]

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [11]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = MIR(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    batch_size_mem=128,
    train_mb_size=128,
    plugins=model_plugins,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
    **best_hyperparameters
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [12]:
for experience in tqdm(train_stream):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/20 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:11<00:00, 41.10it/s]
Epoch 0 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 70.3000
	Loss_MB/train_phase/train_stream/Task000 = 1.2139
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 11.4127
100%|██████████| 469/469 [00:11<00:00, 40.49it/s]
Epoch 1 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 18.4000
	Loss_MB/train_phase/train_stream/Task000 = 0.8489
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0000
	Time_Epoch/train_phase/train_stream/Task000 = 11.5832
100%|██████████| 469/469 [00:12<00:00, 36.29it/s]
Epoch 2 ended.
	CPUUsage_MB/train_phase/train_stream/Task000 = 79.6000
	Loss_MB/train_phase/train_stream/Task000 = 0.7509
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.0001
	Time_Epoch/train_phase/train_stream/Task000 = 12.9228
100%|██████████| 469/469 [00:12<00:00, 37.34it/s]
Epoch 3 ended.
	CPUUsage_MB/train_phase/train_stre

  5%|▌         | 1/20 [02:32<48:14, 152.32s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 44.7757
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x16bbff5b0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9098, 0.1216, 0.1063, 0.1278, 0.1476, 0.0976, 0.1112, 0.1035, 0.1321,
         0.1331, 0.0883, 0.1043, 0.1428, 0.1097, 0.1310, 0.1020, 0.1054, 0.1165,
         0.0707, 0.2126],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.00

 10%|█         | 2/20 [05:25<49:17, 164.32s/it]

...

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 50.6848
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x176f6c9d0>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9098, 0.1216, 0.1063, 0.1278, 0.1476, 0.0976, 0.1112, 0.1035, 0.1321,
         0.1331, 0.0883, 0.1043, 0.1428, 0.1097, 0.1310, 0.1020, 0.1054, 0.1165,
         0.0707, 0.2126],
        [0.8914, 0.9275, 0.1307, 0.0944, 0.1481, 0.0835, 0.0759, 0.0980, 0.1320,
         0.1864, 0.1118, 0.1018, 0.1340, 0.0943, 0.1794, 0.0932, 0.0742, 0.1020,
         0.0976, 0.1409],
        [0.8672, 0.9007, 0.9326, 0.0862, 0.1151, 0.1037, 0.0679, 0.1179, 0.1173,
         0.1616, 0.1127, 0.1106, 0.1399, 0.0775, 0.1738, 0.0760, 0.0688, 0.1243,
         0.1187, 0.0915],
        [0.8216, 0.8757, 0.9139, 0.9356, 0.0754, 0.0729, 0.0867, 0.1237, 0.1040,
         0.1396, 0.1015, 0.1333, 0.1766, 0.0763, 0.18

 40%|████      | 8/20 [26:24<42:45, 213.77s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 42.6789
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x176f98190>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9098, 0.1216, 0.1063, 0.1278, 0.1476, 0.0976, 0.1112, 0.1035, 0.1321,
         0.1331, 0.0883, 0.1043, 0.1428, 0.1097, 0.1310, 0.1020, 0.1054, 0.1165,
         0.0707, 0.2126],
        [0.8914, 0.9275, 0.1307, 0.0944, 0.1481, 0.0835, 0.0759, 0.0980, 0.1320,
         0.1864, 0.1118, 0.1018, 0.1340, 0.0943, 0.1794, 0.0932, 0.0742, 0.1020,
         0.0976, 0.1409],
        [0.8672, 0.9007, 0.9326, 0.0862, 0.1151, 0.1037, 0.0679, 0.1179, 0.1173,
         0.1616, 0.1127, 0.1106, 0.1399, 0.0775, 0.1738, 0.0760, 0.0688, 0.1243,
         0.1187, 0.0915],
        [0.8216, 0.8757, 0.9139, 0.9356, 0.0754, 0.0729, 0.0867, 0.1237, 0.1040,
         0.1396, 0.1015, 0.1333, 0.1766, 0.0763, 0.18

 45%|████▌     | 9/20 [29:47<38:32, 210.20s/it]

-- >> End of eval phase << --
	CPUUsage_Stream/eval_phase/test_stream/Task000 = 44.8950
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x17fcb3e80>
	DiskUsage_Stream/eval_phase/test_stream/Task000 = 2037.1582
	EvalStream/Acc_Matrix = 
tensor([[0.9098, 0.1216, 0.1063, 0.1278, 0.1476, 0.0976, 0.1112, 0.1035, 0.1321,
         0.1331, 0.0883, 0.1043, 0.1428, 0.1097, 0.1310, 0.1020, 0.1054, 0.1165,
         0.0707, 0.2126],
        [0.8914, 0.9275, 0.1307, 0.0944, 0.1481, 0.0835, 0.0759, 0.0980, 0.1320,
         0.1864, 0.1118, 0.1018, 0.1340, 0.0943, 0.1794, 0.0932, 0.0742, 0.1020,
         0.0976, 0.1409],
        [0.8672, 0.9007, 0.9326, 0.0862, 0.1151, 0.1037, 0.0679, 0.1179, 0.1173,
         0.1616, 0.1127, 0.1106, 0.1399, 0.0775, 0.1738, 0.0760, 0.0688, 0.1243,
         0.1187, 0.0915],
        [0.8216, 0.8757, 0.9139, 0.9356, 0.0754, 0.0729, 0.0867, 0.1237, 0.1040,
         0.1396, 0.1015, 0.1333, 0.1766, 0.0763, 0.18

# Get metrics

In [ ]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()
class_accuracies: dict[int, list[float]] = dict()
losses: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    class_accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_ClassAcc_Stream/eval_phase/test_stream/Task000/{i}"
    ][1]
    losses[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Loss_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

In [ ]:
train_cpu_usage = eval_plugin.get_all_metrics()["CPUUsage_MB/train_phase/train_stream/Task000"][1]
train_time_epoch = eval_plugin.get_all_metrics()["Time_Epoch/train_phase/train_stream/Task000"][1]
train_running_time = eval_plugin.get_all_metrics()["RunningTime_Epoch/train_phase/train_stream/Task000"][1]
train_loss_epoch = eval_plugin.get_all_metrics()["Loss_MB/train_phase/train_stream/Task000"][1]

In [ ]:
weights = model_plugins[0].weights

# Store metrics

In [ ]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
    "class_accuracies": class_accuracies,
    "losses": losses,
    "train_cpu_usage": train_cpu_usage,
    "train_time_epoch": train_time_epoch,
    "train_running_time": train_running_time,
    "train_loss_epoch": train_loss_epoch,
    "weights": weights
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))